In [2]:
import pandas as pd

# 项目路径
project_path = '/home/wjunneng/Python/2019-Iflytek-Mobile-Advertising-Anti-Fraud-Algorithm-Challenge'
# round1_iflyad_anticheat_testdata_feature.txt文件路径
testdata_feature_path = project_path + '/data/original/round1_iflyad_anticheat_testdata_feature.txt'
# round1_iflyad_anticheat_traindata.txt文件路径
traindata_path = project_path + '/data/original/round1_iflyad_anticheat_traindata.txt'

def get_testdata_feature(**params):
    """
    返回 testdata_feature 文件内容
    :param params:
    :return:
    """
    testdata_feature_data = pd.read_table(testdata_feature_path, sep='\t')

    return testdata_feature_data


def get_traindata(**params):
    """
    返回 traindata 文件内容
    :param params:
    :return:
    """
    traindata_data = pd.read_table(traindata_path, sep='\t')

    return traindata_data


testdata_feature = get_testdata_feature()
traindata = get_traindata()
traindata.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 29 columns):
sid             1000000 non-null object
label           1000000 non-null int64
pkgname         1000000 non-null object
ver             678176 non-null object
adunitshowid    1000000 non-null object
mediashowid     1000000 non-null object
apptype         1000000 non-null float64
nginxtime       1000000 non-null float64
ip              1000000 non-null object
city            984807 non-null object
province        1000000 non-null float64
reqrealip       1000000 non-null object
adidmd5         1000000 non-null object
imeimd5         1000000 non-null object
idfamd5         1000000 non-null object
openudidmd5     1000000 non-null object
macmd5          1000000 non-null object
dvctype         1000000 non-null float64
model           995533 non-null object
make            876161 non-null object
ntt             1000000 non-null float64
carrier         1000000 non-null float64
os     

In [ ]:
traindata.columns



In [ ]:
# 包名 经过MD5码 加密 【有重复值，有空值，建议one-hot】
print(traindata['pkgname'].unique())
print('\n')

# app版本 【很杂，建议剔除】
print(traindata['ver'].describe())
print('\n')

# 对外广告位 （广告显示的位置）【有重复值，无空值，建议one-hot】
print(traindata['adunitshowid'].describe())
print('\n')

# 对外媒体ID  【有重复值，无空值，建议one-hot】
print(traindata['mediashowid'].describe())
print('\n')




In [9]:
###############################################################################
# app所属分类  【有重复值，无空值，建议one-hot】
print(traindata['apptype'].value_counts())
print('\n')
# app所属分类  【有重复值，无空值，建议one-hot】
print(testdata_feature['apptype'].value_counts())
print('\n')

def deal_apptype(df, **params):
    """
    处理apptype
    :param df: 
    :param params: 
    :return: 
    """
    df['apptype'] = df['apptype'].apply(lambda x: str(x))
    
    return df

# app所属分类  【有重复值，无空值，建议one-hot】
print(traindata['apptype'].value_counts())
print('\n')
# app所属分类  【有重复值，无空值，建议one-hot】
print(testdata_feature['apptype'].value_counts())
print('\n')
###############################################################################



301.0    202453
336.0     94193
326.0     80297
221.0     79289
318.0     72702
          ...  
274.0         1
297.0         1
253.0         1
311.0         1
203.0         1
Name: apptype, Length: 91, dtype: int64




In [ ]:
# 请求到达服务时间
print(traindata['nginxtime'].unique())
print(traindata['nginxtime'].loc[0:3])
print('\n')




In [7]:
# 客户端IP地址
print(traindata['ip'].value_counts())
print('\n')
# 客户端IP地址
print(testdata_feature['ip'].value_counts())
print('\n')



183.224.14.4      226
183.224.14.3      205
183.224.14.14     180
127.0.0.1         147
58.62.203.167     129
                 ... 
110.246.42.41       1
115.51.217.19       1
112.3.70.232        1
60.26.10.198        1
39.181.189.138      1
Name: ip, Length: 747676, dtype: int64


58.62.204.11       55
183.224.14.3       26
183.224.14.4       26
125.77.22.197      17
183.224.14.14      15
                   ..
117.166.216.152     1
27.29.226.9         1
115.63.125.81       1
106.8.250.213       1
106.43.203.112      1
Name: ip, Length: 90931, dtype: int64




In [ ]:
###############################################################################
# 城市
print(traindata['city'].unique())
print('\n')

# 省份
print(traindata['province'].unique())
print('\n')


def deal_city_province(df, **params):
    """
    处理城市和省份
    :param df: 
    :param params: 
    :return: 
    """
    import json
    # 中国省市编码地址
    china_city_list_path = project_path + '/data/foreign/china_city_list.json'
    
    # 省
    provinces = {}
    
    # 市
    cities = {}

    # 读取json
    with open(china_city_list_path, 'r') as file:
        # 读取所有省市数据
        china_city_list = json.load(file)
            
        for province in china_city_list:
            # key:province/value:code
            provinces[province['province']] = province['code']
            
            for city in province['cities']:
                # key:city/value:code
                cities[city['name']] = city['code']
    
    china_city_list = df['city']
    china_city_list = china_city_list.fillna(0)
    
    china_city_list_code = []
    china_province_list_code = []
    
    # 遍历获取code
    for city in china_city_list.values:
        # 是否在provinces中
        if city in dict.fromkeys(provinces.keys(), True):
            tmp = provinces[city]
            china_city_list_code.append(tmp)
            china_province_list_code.append(tmp[:2])
            
        # 是否在cities中
        elif city in dict.fromkeys(cities.keys(), True):
            tmp = cities[city]
            china_city_list_code.append(tmp)
            china_province_list_code.append(tmp[:2])
        
        # 不存在    
        else:
            china_city_list_code.append('100000')
            china_province_list_code.append('10')
            
            if city != 0:
                print(city)
    
    # 替换
    df['city'] = china_city_list_code
    df['province'] = china_province_list_code
                
    return df
    
traindata = deal_city_province(traindata)

# 城市
print(traindata['city'].unique())
print('\n')

# 省份
print(traindata['province'].unique())
print('\n')
###############################################################################




In [ ]:
# 请求的http协议头携带IP，有可能是下游服务器的ip
print(traindata['reqrealip'].unique())
print('\n')




In [ ]:
# Adroid ID的MD5值
print(traindata['adidmd5'].unique())
print('\n')

# imei的MD5值  移动设备国际识别码，是手机设备的唯一识别号码
print(traindata['imeimd5'].unique())
print('\n')




In [ ]:
###############################################################################
# idfa的MD5值
print(traindata['idfamd5'].value_counts())
print('\n')

# idfa的MD5值
print(testdata_feature['idfamd5'].value_counts())
print('\n')

def deal_idfamd5(df, **params):
    """
    处理idfamd5
    :param df: 
    :param params: 
    :return: 
    """    
    del df['idfamd5']
    
    return df
    

# idfa的MD5值
print(traindata['idfamd5'].unique())
print('\n')

# idfa的MD5值
print(testdata_feature['idfamd5'].unique())
print('\n')
###############################################################################



In [ ]:
# openudid的MD5值
print(traindata['openudidmd5'].unique())
print('\n')

# mac的MD5值
print(traindata['macmd5'].unique())
print('\n')




In [ ]:
###############################################################################
# 设备类型 0 – 未知,1 – PC,2 – 手机, 3– 平板,4– 电视盒,5– 智能电视,6 – 可穿戴设备,7 – 智能家电,8 - 音箱,9 - 智能硬件
print(traindata['dvctype'].value_counts())
print('\n')
print(testdata_feature['dvctype'].value_counts())
print('\n')

def deal_dvctype(df, **params):
    """
    返回设备类型
    :param df: 
    :param params: 
    :return: 
    """
    df['dvctype'] = df['dvctype'].apply(lambda x: str(x))
    
    return df

traindata = deal_dvctype(traindata)
testdata_feature = deal_dvctype(testdata_feature)

print(traindata['dvctype'].value_counts())
print('\n')
print(testdata_feature['dvctype'].value_counts())
print('\n')
###############################################################################



In [10]:
# 机型
print(traindata['model'].value_counts())
print('\n')




PBAM00             26578
PACM00             23539
PBEM00             22510
PADM00             14363
VIVO X21A          13293
                   ...  
MEITU%20M4S            1
LGM-X600K              1
OPSSON_V6              1
LG-M250                1
LENOVO A5500-HV        1
Name: model, Length: 7719, dtype: int64




In [11]:
# 厂商
print(traindata['make'].value_counts())
print('\n')




OPPO        250359
vivo        207547
HUAWEI      126195
Xiaomi       48005
HONOR        45321
             ...  
NX541J           1
M2               1
OPPO A33         1
zh-c             1
Leneo            1
Name: make, Length: 2620, dtype: int64




In [ ]:
###############################################################################
# 网络类型 0-未知, 1-有线网, 2-WIFI, 3-蜂窝网络未知, 4-2G, 5-3G, 6–4G
# 构造特征： 0-未知/1-有线网络/2-无线网络
print(traindata['ntt'].value_counts())
print('\n')

print(testdata_feature['ntt'].value_counts())
print('\n')

def deal_ntt(df, **params):
    """
    处理ntt
    :param df: 
    :param params: 
    :return: 
    """
    df['ntt'] = df['ntt'].apply(lambda x: str(x))
    df['ntt_type'] = df['ntt'].apply(lambda x: str(2) if int(float(x)) in [2, 3, 4, 5, 6, 7] else x)
    
    return df

traindata = deal_ntt(traindata)
testdata_feature = deal_ntt(testdata_feature)

print(traindata['ntt'].value_counts())
print('\n')

print(testdata_feature['ntt'].value_counts())
print('\n')

print(traindata['ntt_type'].value_counts())
print('\n')

print(testdata_feature['ntt_type'].value_counts())
print('\n')
###############################################################################




In [ ]:
###############################################################################
# 运营商 0-未知, 46000-移动, 46001-联通, 46003-电信 【-1-中国广电？46002/46005/46006/46007...】
print(traindata['carrier'].value_counts())
print('\n')

print(testdata_feature['carrier'].value_counts())
print('\n')

def deal_carrier(df, **params):
    """
    处理运营商
    :param df: 
    :param params: 
    :return: 
    """
    df['carrier'] = df['carrier'].apply(lambda x: str(1) if int(x) == 0 else x)
    df['carrier'] = df['carrier'].apply(lambda x: str(2) if int(x) == 46000 else x)
    df['carrier'] = df['carrier'].apply(lambda x: str(3) if int(x) == 46001 else x)
    df['carrier'] = df['carrier'].apply(lambda x: str(4) if int(x) == 46003 else x)
    df['carrier'] = df['carrier'].apply(lambda x: str(5) if str(x) not in [str(1), str(2), str(3), str(4)] else x)
    
    return df
    
traindata = deal_carrier(traindata)
testdata_feature = deal_carrier(testdata_feature)
    
# 运营商 0-未知, 46000-移动, 46001-联通, 46003-电信 【-1-中国广电？】
print(traindata['carrier'].value_counts())
print('\n')

print(testdata_feature['carrier'].value_counts())
print('\n')
###############################################################################



In [ ]:
###############################################################################
# 操作系统 : Android, iOS
print(traindata['os'].unique())
print('\n')

# 操作系统 
print(testdata_feature['os'].unique())
print('\n')

def deal_os(df, **params):
    """
    处理操作系统
    :param df: 
    :param params: 
    :return: 
    """
    if 'os' in df.columns:
        # 先将所有值转化为小写
        df['os'] = df['os'].apply(lambda x: x.lower())
        # 如果所有值都相同，则剔除
        if len(set(df['os'])) is 1:
            del df['os']
    
    return df
    
traindata = deal_os(traindata)

# 操作系统
print(traindata.columns)
print('\n')

testdata_feature = deal_os(testdata_feature)

# 操作系统
print(testdata_feature.columns)
print('\n')
###############################################################################




In [12]:
# 操作系统版本
print(traindata['osv'].value_counts())
print('\n')





8.1.0                      302798
9                          142513
6.0.1                       63544
7.1.1                       53333
9.0.0                       48821
                            ...  
5,0                             1
3.2.0-FL2-20180726.9015         1
9.1.0                           1
1.0.0                           1
2.1                             1
Name: osv, Length: 180, dtype: int64




In [ ]:
###############################################################################
# 横竖屏:0竖屏，1横屏
print(traindata['orientation'].unique())
print('\n')

# 横竖屏:0竖屏，1横屏
print(testdata_feature['orientation'].unique())
print('\n')

# train 中orientation为90的仅有一条数据为0，其余结为1 

# (1, 28)
train_2 = traindata[traindata.orientation.astype(int) == 2]
print(train_2.shape)
# (0, 27)
test_2 = testdata_feature[traindata.orientation.astype(int) == 2] 
print(test_2.shape)
# (110, 28)
train_90 = traindata[traindata.orientation.astype(int) == 90] 
print(train_90.shape)
# (15, 27)
test_90 = testdata_feature[traindata.orientation.astype(int) == 90] 
print(test_90.shape)

def deal_orientation(df, **params):
    """
    剔除非0和1的数据 【即非横/竖屏的数据】
    :param df: 
    :param params: 
    :return: 
    """
    df = df[(df['orientation'].astype(int) == 0) | (df['orientation'].astype(int) == 1) | (df['orientation'].astype(int) == 90)]
    
    return df

traindata = deal_orientation(traindata)
testdata_feature = deal_orientation(testdata_feature)

# 横竖屏:0竖屏，1横屏
print(traindata['orientation'].unique())
print('\n')

# 横竖屏:0竖屏，1横屏
print(testdata_feature['orientation'].unique())
print('\n')
###############################################################################




In [ ]:
###############################################################################
# 语言
print(traindata['lan'].value_counts())
print('\n')

print(testdata_feature['lan'].value_counts())
print('\n')

# print(traindata[traindata['lan'] == 'en'][['label', 'province', 'city']])

def deal_lan(df, **params):
    """
    处理语言
    :param df: 
    :param params: 
    :return: 
    """
    df['lan'] = df['lan'].fillna(0)
    
    df['lan'] = df['lan'].apply(lambda x: str(x).lower())
    # 优先处理cn: 1
    df['lan'] = df['lan'].apply(lambda x: '1' if 'cn' in str(x) else x)
    # 处理tw： 2
    df['lan'] = df['lan'].apply(lambda x: '2' if 'tw' in str(x) else x)
    # 处理hk: 2
    df['lan'] = df['lan'].apply(lambda x: '2' if 'hk' in str(x) else x)
    # 处理en: 3
    df['lan'] = df['lan'].apply(lambda x: '3' if 'en' in str(x) else x)
    # 处理zh: 1
    df['lan'] = df['lan'].apply(lambda x: '1' if 'zh' in str(x) else x)
    # 其他： 4
    df['lan'] = df['lan'].apply(lambda x: '4' if not x in ['0', '1', '2', '3'] else x)
    
    return df

# 处理语言
traindata = deal_lan(traindata)
testdata_feature = testdata_feature(testdata_feature)

print(traindata['lan'].value_counts())
print('\n')

print(testdata_feature['lan'].value_counts())
print('\n')
###############################################################################




In [19]:
###############################################################################
# 设备高
print(traindata['h'].value_counts())
print('\n')

# 设备宽
print(traindata['w'].value_counts())
print('\n')

# 屏幕密度
print(traindata['ppi'].value_counts())
print('\n')


def deal_h_w_ppi(df, fillna_type, **params):
    """
    处理宽高/密度
    :param df: 
    :param params: 
    :return: 
    """
    import numpy as np
    
    if fillna_type is 'mean':
        # 利用均值填充
        df['h'].replace(0, df['h'].mean(), inplace=True)
        df['w'].replace(0, df['w'].mean(), inplace=True)
        df['ppi'].replace(0, df['ppi'].mean(), inplace=True)
    
    elif fillna_type is 'median':
        # 利用均值填充
        df['h'].replace(0, df['h'].median(), inplace=True)
        df['w'].replace(0, df['w'].median(), inplace=True)
        df['ppi'].replace(0, df['ppi'].median(), inplace=True)
        
    else:
        # 利用众数填充
        df['h'].replace(0, df['h'].mode().max(), inplace=True)
        df['w'].replace(0, df['w'].mode().max(), inplace=True)
        df['ppi'].replace(0, df['ppi'].mode().max(), inplace=True)
    
    # 添加面积列
    df['area'] = df['h'] * df['w']
    
    # 添加宽高列
    df['aspect_ratio'] = df['h'] / df['w']
    
    # 归一化函数
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

    df['h'] = df[['h']].apply(max_min_scaler)
    df['w'] = df[['w']].apply(max_min_scaler)
    df['ppi'] = df[['ppi']].apply(max_min_scaler)
    
    return df

traindata = deal_h_w_ppi(traindata, 'mean')
testdata_feature = deal_h_w_ppi(testdata_feature, 'mean')


# 设备高
print(traindata['h'].value_counts())
print('\n')

# 设备宽
print(traindata['w'].value_counts())
print('\n')

# 屏幕密度
print(traindata['ppi'].value_counts())
print('\n')
###############################################################################

0.139995    214447
0.212679    134323
0.084118     77156
0.141749     63905
0.259226     58908
             ...  
0.100964         1
0.142192         1
0.145960         1
0.242935         1
0.000000         1
Name: h, Length: 953, dtype: int64


0.114776    419853
0.072779    214440
0.034544    154019
0.074660    125985
0.038222     12651
             ...  
0.130934         1
0.313127         1
0.102741         1
0.037999         1
0.050591         1
Name: w, Length: 441, dtype: int64


0.098387    767256
0.666203    117296
0.002782     47016
0.001391     25973
0.443672     16750
             ...  
0.586926         1
0.383866         1
0.639777         1
0.584145         1
0.545202         1
Name: ppi, Length: 115, dtype: int64


0.139423    214447
0.212155    134323
0.083509     77156
0.141178     63905
0.258734     58908
             ...  
0.071642         1
0.051458         1
0.091827         1
0.052900         1
0.040146         1
Name: h, Length: 953, dtype: int64


0.106613    41